**Collaborative Filtering - User-User and Item-Item Based.**

In [1]:
#import all required packages..

import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats.stats import pearsonr
from sklearn.metrics import pairwise_distances

In [2]:
#import Rating, Movie and Tags Data basically Movie and Tag are just mapping for Movie and Tag
# we are using only Rating Frame as we need to deal with movie and rating with user...

Ratings=pd.read_csv("../input/ratings.csv",encoding="ISO-8859-1")
Movies=pd.read_csv("../input/movies.csv",encoding="ISO-8859-1")
Tags=pd.read_csv("../input/tags.csv",encoding="ISO-8859-1")

In [3]:
Ratings.head()

,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


In [4]:
#Get movie rating count so it will be easy at end to rank movie when 2 movie get same rating....
#this is optional if you want can do it else skip this step.

#1. take groupby with respect to movie and get mean avg of each movie. 
movie_rating_count = Ratings.groupby('movieId')['rating'].count()

#2. convert into dataframe for better operation work.
movie_rating_count = pd.DataFrame(movie_rating_count)

#3. change column name.
movie_rating_count.columns = ['rating_count']

#4. create new column movieIUd
movie_rating_count['movieId'] = movie_rating_count.index
movie_rating_count.reset_index(drop=True)

#merging to Rating dataframe so we have collectively all information together..
Ratings = Ratings.merge(movie_rating_count,right_on='movieId',left_on='movieId')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: 'movieId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


In [5]:
Ratings.head()


,userId,movieId,rating,timestamp,rating_count
0,12882,1,4.0,1147195252,496
1,121987,1,4.5,1308152264,496
2,18127,1,4.0,1130946329,496
3,17318,1,4.0,1168807334,496
4,69519,1,4.0,1303980621,496


In [6]:
Movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
Tags.head()

,movieId,userId,tag,timestamp
0,3916,12882,sports,1147195545
1,4085,12882,Eddie Murphy,1147195966
2,33660,12882,boxing,1147195514
3,1197,320,must show,1145964801
4,1396,320,must show,1145964810


In [8]:
#create a pivot matrix for user and movie based on rating value..
RatingMat = Ratings.pivot_table(index=['userId'],columns=['movieId'],values=['rating'],fill_value=0)

#keep a copy of original matrix for future refrence..
Original_RatingMat = RatingMat.copy()
RatingMat.head()

rating                       ...                              
movieId 1      2      3      4       ...   111362 111759 112556 112852
userId                               ...                              
316        2.5    0.0    0.0    0.0  ...      0.0    0.0    0.0    0.0
320        0.0    0.0    0.0    0.0  ...      0.0    0.0    0.0    0.0
359        5.0    0.0    0.0    0.0  ...      0.0    0.0    0.0    0.0
370        4.5    4.0    0.0    0.0  ...      3.0    4.5    3.5    3.0
910        5.0    4.0    3.5    0.0  ...      0.0    4.5    0.0    0.0

[5 rows x 2500 columns]

In [9]:
#mean normalize to adjust rating on same scale for all user i.e. mean 0 across rows.
# this is because for user1 best mean 5 but user2 best means 4.

RatingMat = RatingMat.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

#drop multilevel column for better usabiltity...
RatingMat.columns = RatingMat.columns.droplevel()
RatingMat.head(5)

movieId,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,31,32,34,35,36,39,41,42,43,44,45,46,...,88140,88163,88744,88810,89492,89745,89864,90866,91500,91529,91542,91630,91658,92259,93840,94864,94959,95510,96079,96610,97304,97752,97913,97921,97938,98809,99114,102125,102445,104841,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,0.46564,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,0.36564,-0.03436,0.46564,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,0.56564,0.36564,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,...,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436,-0.03436
320,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,...,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836,-0.01836
359,0.75440,-0.24560,-0.24560,-0.24560,-0.24560,0.75440,-0.24560,-0.24560,0.55440,0.55440,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,0.35440,-0.24560,-0.24560,0.55440,-0.24560,0.45440,-0.24560,-0.24560,-0.24560,-0.24560,0.15440,-0.24560,0.35440,0.55440,-0.24560,0.35440,-0.24560,-0.24560,0.55440,-0.24560,0.05440,-0.24560,-0.24560,...,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560,-0.24560
370,0.77524,0.67524,-0.12476,-0.12476,-0.12476,0.87524,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,0.77524,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,0.87524,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,-0.12476,...,0.47524,0.67524,-0.12476,-0.12476,0.67524,0.37524,-0.12476,-0.12476,0.57524,-0.12476,-0.12476,-0.12476,-0.12476,0.37524,0.77524,0.07524,-0.12476,-0.12476,0.67524,0.67524,-0.12476,-0.12476,0.57524,0.67524,-0.12476,0.67524,0.77524,0.17524,-0.12476,-0.12476,0.37524,0.47524,0.77524,0.67524,-0.12476,-0.12476,0.47524,0.77524,0.57524,0.47524
910,0.78888,0.58888,0.48888,-0.21112,0.48888,0.48888,-0.21112,-0.21112,-0.21112,0.58888,-0.21112,-0.21112,-0.21112,0.58888,0.58888,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,0.38888,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,0.58888,0.38888,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,-0.21112,...,0.48888,0.18888,-0.21112,0.68888,0.68888,-0.21112,-0.21112,0.58888,-0.21112,-0.21112,0.48888,-0.21112,0.58888,-0.21112,-0.21112,-0.21112,-0.21112,0.48888,0.48888,0.18888,0.38888,-0.21112,-0.21112,0.58888,-0.21112,0.38888,-0.21112,-0.21112,0.48888,-0.21112,-0.21112,-0.21112,0.48888,-0.21112,-0.21112,-0.21112,-0.

In [10]:

#user wise similarity with cosine...
user_similarity = cosine_similarity(RatingMat)
user_sim_df = pd.DataFrame(user_similarity,index=RatingMat.index,columns=RatingMat.index)
user_sim_df.head(5)

#we can do it using pearson correlation as well there is no hard and fast rule.
# generally user-user approach we use pearson correlation and item-item we use cosine similarity


#user wise similarity with pearson coorelation...
# user_similarity = 1-pairwise_distances(RatingMat, metric="correlation")
# user_sim_df = pd.DataFrame(user_similarity,index=RatingMat.index,columns=RatingMat.index)
# user_sim_df.head(5)


userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
316,1.000000,0.299545,0.225689,0.215937,0.171846,0.229365,0.247069,0.210951,0.007233,0.206959,0.210236,0.165468,0.263808,0.351433,0.271860,0.403737,0.297588,0.133888,0.283641,0.188762,0.092259,0.335727,0.095579,0.298077,0.008331,0.100187,0.191573,0.191049,0.253549,0.280088,0.272902,0.037400,0.296571,0.285572,0.260100,0.274855,0.042831,0.310526,0.214375,0.335500,...,0.229131,0.327392,0.177044,0.249211,0.124807,0.198872,0.211552,0.145529,0.261131,0.186477,0.118848,0.163399,0.227150,0.325500,0.078195,0.198481,0.299620,0.088436,0.288714,0.139964,0.268222,0.247878,0.339692,0.221556,0.272798,0.255439,0.164336,0.104490,0.231532,-0.037684,0.290918,0.240003,0.035299,0.215968,0.367558,0.045559,0.278043,0.193846,0.017233,0.149235
320,0.299545,1.000000,0.183217,0.187426,0.157678,0.242885,0.160790,0.184277,-0.014179,0.197965,0.267901,0.106024,0.231658,0.237701,0.209262,0.284997,0.238501,0.115835,0.190973,0.229348,0.094611,0.270563,0.107911,0.238769,0.094408,0.134041,0.181234,0.213788,0.198654,0.212034,0.198593,0.029766,0.214467,0.225539,0.301686,0.316056,0.013472,0.259090,0.249841,0.228808,...,0.238855,0.240127,0.118646,0.151526,0.186118,0.193069,0.174965,0.110303,0.172660,0.155367,0.057378,0.128877,0.154723,0.220365,0.045218,0.244092,0.191717,0.139516,0.248366,0.168141,0.245971,0.252219,0.275256,0.184093,0.267175,0.207609,0.098373,0.199685,0.139690,0.002081,0.229539,0.323584,0.123649,0.170613,0.273515,0.067225,0.195021,0.180317,0.037405,0.096617
359,0.225689,0.183217,1.000000,0.218920,0.188056,0.239563,0.210592,0.168052,0.058551,0.252456,0.199123,0.100695,0.204331,0.269093,0.300624,0.279574,0.259844,0.110061,0.326794,0.244947,0.093673,0.306212,0.091438,0.218337,0.103668,0.126514,0.228987,0.219236,0.239594,0.193876,0.228493,0.052147,0.274040,0.327850,0.208851,0.205021,0.016990,0.248523,0.134581,0.267614,...,0.231296,0.172331,0.220817,0.243449,0.127993,0.208144,0.160910,0.094889,0.108024,0.148938,0.100218,0.279385,0.247512,0.278444,0.055333,0.223439,0.184431,0.169374,0.267785,0.192359,0.209124,0.169292,0.261677,0.212649,0.246712,0.279577,0.156561,0.076375,0.262403,0.008070,0.244404,0.190439,0.010915,0.173494,0.209795,0.011065,0.205926,0.120846,0.053407,0.142030
370,0.215937,0.187426,0.218920,1.000000,0.180415,0.323008,0.142359,0.136277,0.095462,0.291159,0.215419,0.186484,0.158642,0.256498,0.318828,0.266846,0.342799,0.092376,0.204237,0.293971,0.330903,0.265019,0.049601,0.238441,0.057697,0.099174,0.268694,0.188885,0.302884,0.247241,0.243267,0.065397,0.257333,0.317108,0.258303,0.355849,0.123117,0.311658,0.187185,0.192698,...,0.146021,0.170656,0.249314,0.126736,0.111530,0.378073,0.093593,0.122757,0.196614,0.191125,0.206481,0.181137,0.180508,0.218467,0.077334,0.230126,0.165477,0.117924,0.234430,0.290348,0.302625,0.167190,0.315646,0.123945,0.285167,0.335208,0.296905,0.154279,0.207108,0.016908,0.279538,0.166676,0.102677,0.220298,0.228517,0.061691,0.272535,0.079117,0.050965,0.127138
910,0.171846,0.157678,0.188056,0.180415,1.000000,0.257296,0.209615,0.192566,0.040182,0.310700,0.107393,0.147180,0.144108,0.191550,0.157343,0.171393,0.199123,0.124323,0.194183,0.203142,0.145148,0.194243,0.124409,0.170876,0.033195,0.171790,0.234380,0.116248,0.166320,0.194630,0.206961,0.067636,0.214287,0.221083,0.183166,0.227179,0.108781,0.196836,0.091704,0.203452,...,0.252968,0.114237,0.175134,0.246031,0.139158,0.205584,0.120070,0.082000,0.151083,0.149750,0.069493,0.255281,0.189282,0.224824,0.068023,0.183970,0

In [11]:
# 1. select any user randomly for example we have selected userid 359 
#Get a list of movie user already watched so we can remove from our recommendation list..

curr_user_rated_movie = Ratings[(Ratings.userId == 359) & (Ratings.rating != 0)]['movieId']
curr_user_rated_movie = pd.DataFrame(curr_user_rated_movie,columns=['movieId'])

In [12]:
# put similarity of current user i.e. 359 in a dataframe because later we need for weighted average..

curr_user_similarity = pd.DataFrame(user_sim_df.loc[359])

#just changing column name as similarity for better readability.
curr_user_similarity.rename(columns={359:'similarity'},inplace=True)
curr_user_similarity.head()

,similarity
userId,
316,0.225689
320,0.183217
359,1.000000
370,0.218920
910,0.188056


In [13]:
# Remove movieId which user already watched from our rating matrix..
# above we have calculated which movie user has watched.
# subtract or negeate from rating matrix we get still not watched movie..

RatingMatTranspose = RatingMat.T
RatingMatTranspose = RatingMatTranspose.loc[~RatingMatTranspose.index.isin(curr_user_rated_movie.movieId.tolist())]

In [14]:
RatingMatTranspose.head()

userId,316,320,359,370,910,975,1015,1387,1447,1588,1738,1741,1835,1935,2024,2248,2702,2847,2945,2988,3029,3076,3569,3665,3915,4215,4347,4483,4629,4931,5063,5268,5450,5475,5611,6116,6937,6988,7541,7723,...,131630,131835,131910,132187,132229,132295,132470,132578,132595,132914,133047,133811,134156,134181,134196,134279,134316,134401,134521,134525,134557,134627,134964,135092,135223,135418,136015,136037,136455,136856,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,-0.03436,-0.01836,-0.2456,0.67524,0.58888,-0.076489,0.51984,0.54344,-0.006622,-0.15832,-0.02716,0.64432,0.73604,-0.07144,0.55756,-0.06304,-0.11192,-0.04268,-0.21528,0.31592,-0.0802,-0.08096,-0.02356,-0.05476,-0.05976,-0.11828,-0.24096,-0.12968,0.38104,-0.08348,0.45072,-0.00628,-0.10168,-0.1958,-0.05132,-0.0564,-0.02684,-0.11984,-0.01836,-0.15448,...,0.23312,-0.01016,-0.21956,-0.30388,-0.05432,-0.10728,-0.0414,-0.00968,0.639,0.45892,-0.05872,0.46884,0.33988,0.36688,-0.01044,-0.06188,-0.03156,0.24808,-0.08948,0.6704,-0.0594,-0.06968,-0.0914,-0.15972,0.11068,0.56432,0.46744,-0.00836,0.57244,-0.02096,-0.13564,0.66612,-0.010178,0.66468,-0.03116,-0.00772,0.2706,0.05636,-0.0046,-0.11444
3,-0.03436,-0.01836,-0.2456,-0.12476,0.48888,-0.076489,-0.08016,-0.05656,-0.006622,-0.15832,-0.02716,-0.15568,-0.06396,-0.07144,-0.14244,-0.06304,-0.11192,-0.04268,-0.21528,-0.18408,-0.0802,-0.08096,0.37644,-0.05476,-0.05976,-0.11828,-0.24096,-0.12968,-0.11896,-0.08348,0.55072,-0.00628,-0.10168,-0.1958,-0.05132,-0.0564,-0.02684,-0.11984,-0.01836,-0.15448,...,-0.16688,-0.01016,-0.21956,-0.30388,-0.05432,-0.10728,-0.0414,-0.00968,-0.061,-0.14108,-0.05872,0.16884,0.33988,-0.23312,-0.01044,-0.06188,-0.03156,-0.25192,-0.08948,-0.1296,-0.0594,-0.06968,-0.0914,-0.15972,-0.08932,0.36432,-0.13256,-0.00836,0.57244,-0.02096,-0.13564,-0.03388,-0.010178,-0.03532,-0.03116,-0.00772,-0.1294,0.05636,-0.0046,-0.11444
4,-0.03436,-0.01836,-0.2456,-0.12476,-0.21112,-0.076489,-0.08016,-0.05656,-0.006622,-0.15832,-0.02716,-0.15568,-0.06396,-0.07144,-0.14244,-0.06304,-0.11192,-0.04268,-0.21528,-0.18408,-0.0802,-0.08096,-0.02356,-0.05476,-0.05976,-0.11828,-0.24096,-0.12968,-0.11896,-0.08348,-0.14928,-0.00628,-0.10168,-0.1958,-0.05132,-0.0564,-0.02684,-0.11984,-0.01836,-0.15448,...,-0.16688,-0.01016,-0.21956,-0.30388,-0.05432,-0.10728,-0.0414,-0.00968,-0.061,-0.14108,-0.05872,-0.53116,-0.26012,-0.23312,-0.01044,-0.06188,-0.03156,-0.25192,-0.08948,-0.1296,-0.0594,-0.06968,-0.0914,-0.15972,-0.08932,-0.23568,-0.13256,-0.00836,-0.12756,-0.02096,-0.13564,-0.03388,-0.010178,-0.03532,-0.03116,-0.00772,-0.1294,-0.14364,-0.0046,-0.11444
5,-0.03436,-0.01836,-0.2456,-0.12476,0.48888,-0.076489,0.61984,-0.05656,-0.006622,-0.15832,-0.02716,-0.15568,-0.06396,-0.07144,-0.14244,-0.06304,-0.11192,-0.04268,-0.21528,-0.18408,-0.0802,-0.08096,-0.02356,-0.05476,-0.05976,-0.11828,-0.24096,-0.12968,-0.11896,-0.08348,-0.14928,-0.00628,-0.10168,-0.1958,-0.05132,-0.0564,-0.02684,-0.11984,-0.01836,-0.15448,...,0.23312,-0.01016,-0.21956,0.49612,-0.05432,-0.10728,-0.0414,-0.00968,-0.061,-0.14108,-0.05872,-0.53116,-0.26012,0.26688,-0.01044,-0.06188,-0.03156,-0.25192,-0.08948,-0.1296,-0.0594,-0.06968,-0.0914,-0.15972,-0.08932,-0.23568,-0.13256,-0.00836,-0.12756,-0.02096,-0.13564,-0.03388,-0.010178,-0.03532,-0.03116,-0.00772,-0.1294,-0.14364,-0.0046,-0.11444
7,0.36564,-0.01836,-0.2456,-0.12476,-0.21112,-0.076489,-0.08016,-0.05656,-0.006622,-0.15832,-0.02716,0.44432,-0.06396,-0.07144,-0.14244,-0.06304,-0.11192,-0.04268,-0.21528,-0.18408,-0.0802,-0.08096,-0.02356,-0.05476,-0.05976,-0.11828,-0.24096,-0.12968,-0.11896,-0.08348,-0.14928,-0.00628,-0.10168,-0.1958,-0.05132,-0.0564,-0.02684,-0.11984,-0.01836,-0.15448,...,0.43312,-0.01016,0.48044,0.69612,-0.05432,-0.10728,-0.0414,-0.00968,-0.061,-0.14108,-0.05872,0.46884,0.23988,-0.23312,-0.01044,-0.06188,-0.03156,-0.25192,-0.08948,-0.1296,-0.0594,-0.06968,-0.0914,-0.15972,-0.08932,-0.23568,-0.13256,-0.

In [15]:
#when we have taken pivot table it created multi level column name , so drop for better understanding..

Original_RatingMat.columns = Original_RatingMat.columns.droplevel()

In [16]:
'''
calculate the weighted average a movie per user so later we can show a user k specific - 
avg specific predicition rating for a movie

formulas :- sum(w[1]*user1+w[2]*user2.....w[n]*userN) / w[1]+w[2]....w[n] 
'''           
Weighed_avg = []

for movieId in RatingMatTranspose.index:
    
    '''
     User not watched movie consider as 0
     we need to remove those weight where user has not watched movie in weighted sum.
     In weighted sum in denominator we need to sum all weights
     we will remove weights for those rating is 0.
    '''
    
    user_not_rated = Original_RatingMat[Original_RatingMat[movieId] == 0].index
    
    #calculating weights.
    Total_weight = np.sum(curr_user_similarity.loc[~curr_user_similarity.index.isin(user_not_rated.tolist())]['similarity'])
    
    #appending in weighted_avg list to get final list of weighted avg movie.
    Weighed_avg.append(np.dot(RatingMatTranspose.loc[movieId],curr_user_similarity.similarity) / (Total_weight))

In [17]:
#converting Weighted list in dataframe for better operation works..

Weighed_avg = pd.DataFrame(Weighed_avg,columns=['weighted_avg'])
Weighed_avg.index = RatingMatTranspose.index

In [18]:
#Weighed_avg = Weighed_avg.loc[~Weighed_avg.index.isin(curr_user_rated_movie.movieId.tolist())]

#sort top 10 recommendation movie based on weighted avg here you can append movie count as well if you want.
Weighed_avg.sort_values(by='weighted_avg',ascending=False).head(10)
top_recommendation = Weighed_avg.sort_values(by='weighted_avg',ascending=False)

In [19]:
#validate this manually by seeing past history of current user and recommended movie list..
# as we see below current user mostly love to watch comedy/drama and other 

#first merge MovieId in movie name and genre so we can classify based on genre and name..
Rating_Movie = Ratings.merge(Movies, left_on='movieId',right_on='movieId')
Watched_Genre = pd.DataFrame(Rating_Movie[Rating_Movie.userId==359].groupby('genres').size(),columns=['Count'])
Watched_Genre.sort_values(by='Count',ascending=False).head(10)

,Count
genres,
Drama,41
Comedy,34
Comedy|Drama|Romance,24
Comedy|Drama,24
Comedy|Romance,20
Drama|Romance,19
Drama|Thriller,19
Crime|Drama,17
Action|Adventure|Sci-Fi,16


In [20]:
#our recommendation is matching with user past history ...
top_recommendation['movieId'] = top_recommendation.index
Movies_recommended = Movies.merge(top_recommendation, left_on='movieId',right_on='movieId')
Watched_recommended_Genre = pd.DataFrame(Movies_recommended.groupby('genres').size(),columns=['Count'])
Watched_recommended_Genre.sort_values(by='Count',ascending=False).head(10)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: 'movieId' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Count
genres,
Comedy,163
Drama,160
Comedy|Drama,78
Drama|Romance,74
Comedy|Romance,74
Comedy|Drama|Romance,59
Drama|Thriller,36
Comedy|Crime,25
Crime|Drama,23
